Описание задания:
В домашнем задании нужно решить задачу классификации наличия болезни сердца у пациентов наиболее эффективно. Данные для обучения моделей необходимо загрузить самостоятельно с сайта. Целевая переменная – наличие болезни сердца (HeartDisease). Она принимает значения 0 или 1 в зависимости от отсутствия или наличия болезни соответственно. Подробное описание признаков можно прочесть в описании датасета на сайте. Для выполнения работы не обязательно вникать в медицинские показатели.

Этапы работы:

    Получите данные и загрузите их в рабочую среду. (Jupyter Notebook или другую)

    Подготовьте датасет к обучению моделей:

    a) Категориальные переменные переведите в цифровые значения. Можно использовать pd.get_dummies, preprocessing.LabelEncoder. Старайтесь не использовать для этой задачи циклы.

    Разделите выборку на обучающее и тестовое подмножество. 80% данных оставить на обучающее множество, 20% на тестовое.

    Обучите модель логистической регрессии с параметрами по умолчанию.
    Подсчитайте основные метрики модели. Используйте следующие метрики и функцию:

    cross_validate(…, cv=10, scoring=['accuracy','recall','precision','f1'])

    Оптимизируйте 3-4 параметра модели:

    a) Используйте GridSearchCV.

    b) Используйте RandomizedSearchCV.

    c) *Добавьте в п. 6b 2-5 моделей классификации и вариации их параметров.

    d) Повторите п. 5 после каждого итогового изменения параметров.

    Сформулируйте выводы по проделанной работе:

    a) Сравните метрики построенных моделей.

    b) *Сравните с полученными результатами в домашнем задании по теме 
    
    «Ансамблирование».


In [115]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import  BaggingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifierCV
from sklearn.metrics import classification_report as c_rep
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_validate
from scipy.stats import uniform

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [116]:
heart_str = '/content/drive/MyDrive/Netology_homeworks/06Machine_Learning/09.Ansambling/heart.csv'

In [117]:
heart_data = pd.read_csv(heart_str)

In [118]:
y_data = heart_data['HeartDisease']

In [119]:
heart_data_copy = pd.get_dummies(heart_data.copy().drop('HeartDisease', axis=1), columns=['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope'])

In [120]:
heart_data_copy.head()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,Sex_F,Sex_M,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ExerciseAngina_N,ExerciseAngina_Y,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,40,140,289,0,172,0.0,0,1,0,1,0,0,0,1,0,1,0,0,0,1
1,49,160,180,0,156,1.0,1,0,0,0,1,0,0,1,0,1,0,0,1,0
2,37,130,283,0,98,0.0,0,1,0,1,0,0,0,0,1,1,0,0,0,1
3,48,138,214,0,108,1.5,1,0,1,0,0,0,0,1,0,0,1,0,1,0
4,54,150,195,0,122,0.0,0,1,0,0,1,0,0,1,0,1,0,0,0,1


In [121]:
heart_data_copy.columns

Index(['Age', 'RestingBP', 'Cholesterol', 'FastingBS', 'MaxHR', 'Oldpeak',
       'Sex_F', 'Sex_M', 'ChestPainType_ASY', 'ChestPainType_ATA',
       'ChestPainType_NAP', 'ChestPainType_TA', 'RestingECG_LVH',
       'RestingECG_Normal', 'RestingECG_ST', 'ExerciseAngina_N',
       'ExerciseAngina_Y', 'ST_Slope_Down', 'ST_Slope_Flat', 'ST_Slope_Up'],
      dtype='object')

In [122]:
X_train, X_test, y_train, y_test = train_test_split(heart_data_copy, y_data, test_size=0.2, random_state=42)

In [123]:
logReg = LogisticRegression(random_state=42, max_iter=1000)

In [124]:
logReg.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [125]:
logreg_cv = cross_validate(logReg, X=X_test, y=y_test, cv=10, scoring=['accuracy','recall','precision','f1'])

In [126]:
logistic = LogisticRegression(solver='saga', tol=1e-2, max_iter=200,
                              random_state=0)
distributions = dict(C=uniform(loc=0, scale=4),
                     penalty=['l2', 'l1'])
clf = RandomizedSearchCV(logistic, distributions, random_state=0)
clf.fit(X_train, y_train)

RandomizedSearchCV(estimator=LogisticRegression(max_iter=200, random_state=0,
                                                solver='saga', tol=0.01),
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7f17a7cade80>,
                                        'penalty': ['l2', 'l1']},
                   random_state=0)

In [127]:
logistic.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 200,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': 0,
 'solver': 'saga',
 'tol': 0.01,
 'verbose': 0,
 'warm_start': False}

In [128]:
rand_search_cv = cross_validate(clf, X=X_test, y=y_test, cv=10, scoring=['accuracy','recall','precision','f1'])

In [129]:
parameters = {'multi_class':('auto', 'ovr', 'multinomial'), 'verbose': [0, 1, 2]}

In [130]:
grid_search = GridSearchCV(logistic, parameters)

In [131]:
grid_search_cv = cross_validate(grid_search, X=X_test, y=y_test, cv=10, scoring=['accuracy','recall','precision','f1'])

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jo

convergence after 51 epochs took 0 seconds
convergence after 49 epochs took 0 seconds
convergence after 48 epochs took 0 seconds
convergence after 46 epochs took 0 seconds
convergence after 39 epochs took 0 seconds
convergence after 51 epochs took 0 seconds
convergence after 49 epochs took 0 seconds
convergence after 48 epochs took 0 seconds
convergence after 46 epochs took 0 seconds
convergence after 39 epochs took 0 seconds
convergence after 51 epochs took 0 seconds
convergence after 49 epochs took 0 seconds


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Par

convergence after 48 epochs took 0 seconds
convergence after 46 epochs took 0 seconds
convergence after 39 epochs took 0 seconds
convergence after 51 epochs took 0 seconds
convergence after 49 epochs took 0 seconds
convergence after 48 epochs took 0 seconds
convergence after 46 epochs took 0 seconds
convergence after 39 epochs took 0 seconds


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jo

...

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent work

convergence after 33 epochs took 0 seconds
convergence after 34 epochs took 0 seconds
convergence after 31 epochs took 0 seconds
convergence after 20 epochs took 0 seconds
convergence after 24 epochs took 0 seconds
convergence after 46 epochs took 0 seconds
convergence after 46 epochs took 0 seconds
convergence after 43 epochs took 0 seconds
convergence after 28 epochs took 0 seconds
convergence after 31 epochs took 0 seconds
convergence after 46 epochs took 0 seconds
convergence after 46 epochs took 0 seconds
convergence after 43 epochs took 0 seconds


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Para

convergence after 28 epochs took 0 seconds
convergence after 31 epochs took 0 seconds
convergence after 46 epochs took 0 seconds
convergence after 46 epochs took 0 seconds
convergence after 43 epochs took 0 seconds
convergence after 28 epochs took 0 seconds
convergence after 31 epochs took 0 seconds
convergence after 46 epochs took 0 seconds
convergence after 46 epochs took 0 seconds
convergence after 43 epochs took 0 seconds
convergence after 28 epochs took 0 seconds
convergence after 31 epochs took 0 seconds
convergence after 35 epochs took 0 seconds
convergence after 36 epochs took 0 seconds
convergence after 34 epochs took 0 seconds
convergence after 57 epochs took 0 seconds
convergence after 24 epochs took 0 seconds
convergence after 35 epochs took 0 seconds
convergence after 36 epochs took 0 seconds
convergence after 34 epochs took 0 seconds
convergence after 57 epochs took 0 seconds
convergence after 24 epochs took 0 seconds


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jo

Данные, полученные из раздела "Ансамблирование" (LinearSVC у меня давал ошибку, поэтому пришлось его закомментировать):

In [132]:
stacking_classifier = StackingClassifier(
    [
      # ('LinearSVC', LinearSVC(random_state=42, max_iter=50)),
      ('DecisionTreeClassifier', DecisionTreeClassifier(random_state=42)),
      ('RandomForest', RandomForestClassifier(random_state=42)) 
    ],
    RidgeClassifierCV([8,14,10])
)

In [133]:
stacking_classifier.fit(X_train, y_train)

StackingClassifier(estimators=[('DecisionTreeClassifier',
                                DecisionTreeClassifier(random_state=42)),
                               ('RandomForest',
                                RandomForestClassifier(random_state=42))],
                   final_estimator=RidgeClassifierCV(alphas=[8, 14, 10]))

In [134]:
ensemble_cv = cross_validate(stacking_classifier, X=X_test, y=y_test, cv=10, scoring=['accuracy','recall','precision','f1'])

In [140]:
cols = np.arange(0,10)
index = ['LogisticRegression', 'RandomizedSearchCV', 'GridSearchCV', 'StackingClassifier']

In [141]:
df = pd.DataFrame([logreg_cv['test_accuracy'], rand_search_cv['test_accuracy'], grid_search_cv['test_accuracy'], ensemble_cv['test_accuracy']], columns=cols, index=index)

In [146]:
print(df.max(axis=1))

LogisticRegression    1.000000
RandomizedSearchCV    0.888889
GridSearchCV          0.888889
StackingClassifier    0.947368
dtype: float64
